# 2005-2019 VA House of Delegates Election

## Election Results

These precinct-level election results come directly from the Virginia Department of Elections, and they require significant cleaning. 

_I've tested this cleaning method on 2011 data, now I'm going to go through and run it on the data from 2005-2019_. 

In [1]:
library(sf)
library(ggplot2)
library(dplyr)
library(tibble)
library(magrittr)


files <- list.files("C:/Users/madie/OneDrive/data/official-VA-2005-2019/", full.names = TRUE)
df <- setNames(lapply(files, read.csv), tools::file_path_sans_ext(basename(files)))
names(df)

Linking to GEOS 3.8.0, GDAL 3.0.4, PROJ 6.3.1


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




[1] "G05" "G07" "G09" "G11" "G13" "G15" "G17" "G19"

Now, this csv file includes the results by precinct for every single election that took place in Virginia in 2011 at 11, so I have some serious filtering to do. Currently, there are around 57,000 records. 

Filters:
- `DistrictType` = "House of Delegates" -> 7002 records
- `Party` = Democratic or Republican (sorry third parties) -> 3868 records
- `PrecinctName` != "# AB - Central Absentee Precinct" or "## Provisional" -> 3376 records
    - Provisional ballots and absentee ballots aren't assigned a precinct, so I can't use them to measure precinct-level election results

In [42]:
#df <- df[df$DistrictType == "House of Delegates",]

clear_non_precinct <- function(x) {
    x %>% 
        filter(DistrictType == "House of Delegates") %>%
        filter(Party %in% c("Democratic", "Republican")) %>%
        filter(!(PrecinctName %in% c("# AB - Central Absentee Precinct", "## Provisional")))
}

df <- df %>% lapply(clear_non_precinct)
nrow(df$G05)

[1] 3148

In [43]:
names(df$G05)

[1] "CandidateUid" "FirstName"    "MiddleName"   "LastName"     "Suffix"      
 [6] "TOTAL_VOTES"  "Party"        "WriteInVote"  "LocalityUid"  "LocalityCode"
[11] "LocalityName" "PrecinctUid"  "PrecinctName" "DistrictUid"  "DistrictType"
[16] "DistrictName" "OfficeUid"    "OfficeTitle"  "ElectionUid"  "ElectionType"
[21] "ElectionDate" "ElectionName"

Now I have 3148 records in the first dataframe (2005), where each record is one candidate running in one precinct. What I would like to do is produce a pivot table, where:
- index = `PrecinctName`
- columns
    - `GyyDHOD` = all votes for Democratic candidates in that precinct, where `yy` is the year
    - `GyyRHOD` = all votes for Republican candidates in that precinct

In [44]:
join_votes <- function(x) {
    x %>%
        group_by(PrecinctName, LocalityName) %>%
        summarise(DHOD = sum(TOTAL_VOTES[Party == "Democratic"]),
                  RHOD = sum(TOTAL_VOTES[Party == "Republican"])) %>%
        distinct()
}

df_votes <- df %>% lapply(join_votes)

print(df_votes$G17)



`summarise()` regrouping output by 'PrecinctName' (override with `.groups` argument)

`summarise()` regrouping output by 'PrecinctName' (override with `.groups` argument)

`summarise()` regrouping output by 'PrecinctName' (override with `.groups` argument)

`summarise()` regrouping output by 'PrecinctName' (override with `.groups` argument)

`summarise()` regrouping output by 'PrecinctName' (override with `.groups` argument)

`summarise()` regrouping output by 'PrecinctName' (override with `.groups` argument)

`summarise()` regrouping output by 'PrecinctName' (override with `.groups` argument)

`summarise()` regrouping output by 'PrecinctName' (override with `.groups` argument)



# A tibble: 2,439 x 4
# Groups:   PrecinctName [2,380]
   PrecinctName                     LocalityName           DHOD  RHOD
   <chr>                            <chr>                 <int> <int>
 1 "001 - ARLINGTON"                ARLINGTON COUNTY        962   206
 2 "001 - CENTRAL"                  POQUOSON CITY           251  1125
 3 "001 - CHESAPEAKE"               MATHEWS COUNTY          234   545
 4 "001 - DEAN"                     MANASSAS CITY           696   401
 5 "001 - EAST"                     GALAX CITY                0   441
 6 "001 - EAST PRECINCT"            RADFORD CITY            862   457
 7 "001 - EAST WARD"                NORTON CITY             295   557
 8 "001 - EMANUEL A. M. E. CHURCH " PORTSMOUTH CITY         806     0
 9 "001 - FIRST"                    COLONIAL HEIGHTS CITY   195   427
10 "001 - FIRST WARD"               BRISTOL CITY              0   535
# ... with 2,429 more rows


In [45]:
df_votes$G17 <- df_votes$G17 %>% separate(col = PrecinctName, sep = "-", into = c("district", "precinct"), extra="merge")
df_votes$G17$precinct <- df_votes$G17$precinct %>% trimws(which = c("both"))
write.csv(df_votes$G17, "2017-precinct-results.csv")

Warning message:
"Expected 2 pieces. Missing pieces filled with `NA` in 2 rows [978, 2168]."


In [4]:
names(df_votes)

[1] "G05" "G07" "G09" "G11" "G13" "G15" "G17" "G19"

In [5]:
# TODO: Do this with a loop. I know this is bad, but I can't figure out how else to do it. 
df_votes$G05 <- df_votes$G05 %>% setNames(nm = c("PrecinctName", "G05DHOD", "G05RHOD"))
df_votes$G07 <- df_votes$G07 %>% setNames(nm = c("PrecinctName", "G07DHOD", "G07RHOD"))
df_votes$G09 <- df_votes$G09 %>% setNames(nm = c("PrecinctName", "G09DHOD", "G09RHOD"))
df_votes$G11 <- df_votes$G11 %>% setNames(nm = c("PrecinctName", "G11DHOD", "G11RHOD"))
df_votes$G13 <- df_votes$G13 %>% setNames(nm = c("PrecinctName", "G13DHOD", "G13RHOD"))
df_votes$G15 <- df_votes$G15 %>% setNames(nm = c("PrecinctName", "G15DHOD", "G15RHOD"))
df_votes$G17 <- df_votes$G17 %>% setNames(nm = c("PrecinctName", "G17DHOD", "G17RHOD"))
df_votes$G19 <- df_votes$G19 %>% setNames(nm = c("PrecinctName", "G19DHOD", "G19RHOD"))
print(df_votes)

$G05
# A tibble: 2,178 x 3
   PrecinctName                     G05DHOD G05RHOD
   <chr>                              <int>   <int>
 1 "001 - ARLINGTON"                    616       0
 2 "001 - CENTRAL"                        0     773
 3 "001 - CHESAPEAKE"                     0     569
 4 "001 - DEAN"                         497    1090
 5 "001 - EAST"                           0     406
 6 "001 - EAST WARD"                    400     164
 7 "001 - EAST WARD PRECINCT"           457     530
 8 "001 - EMANUEL A. M. E. CHURCH "     701       0
 9 "001 - FIRST"                          0     643
10 "001 - FIRST WARD"                  1668     524
# ... with 2,168 more rows

$G07
# A tibble: 2,266 x 3
   PrecinctName                     G07DHOD G07RHOD
   <chr>                              <int>   <int>
 1 "001 - ARLINGTON"                    349       0
 2 "001 - CENTRAL"                        0     627
 3 "001 - CHESAPEAKE"                     0     704
 4 "001 - DEAN"                   

Ok, so now I've calculated the votes for the respective candidates by precinct. The next step will be to add the population and voting-age population by precinct, using the IPUMS. 

## Matching to shapefiles and demographic data

I'm going to put a pause on aquiring the most accurate demographic data for each year, since that is more of a "nice to have," and instead focus on adding in the shapefiles. For that, I'm using the "VA_precincts" file prepared by MGGG, since it already matches demographic data to precincts. 

To pair them together, I've noticed that the field `precinct` in "VA_precincts" and the text component of `PrecinctName` seem to match up one-to-one. 

The first thing I need to do is split up the current field called `PrecinctName` into `precinctID` and `precinct`. 

In [6]:
library(tidyverse)

split_precinct <- function(x) {
    x[order(x$PrecinctName),] %>%
        separate(col = PrecinctName, sep = " - ", into = c("precinctID", "precinct"))
}
df_votes <- df_votes %>% lapply(split_precinct)
print(df_votes)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.0
v purrr   0.3.4     

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x tidyr::extract()   masks magrittr::extract()
x dplyr::filter()    masks stats::filter()
x dplyr::lag()       masks stats::lag()
x purrr::set_names() masks magrittr::set_names()

Warning message:
"Expected 2 pieces. Additional pieces discarded in 6 rows [400, 739, 1068, 1383, 1446, 1457]."
Warning message:
"Expected 2 pieces. Missing pieces filled with `NA` in 6 rows [2173, 2174, 2175, 2176, 2177, 2178]."
Warning message:
"Expected 2 pieces. Additional pieces discarded in 6 rows [415, 767, 1106, 1439, 1505, 1517]."
Warning message:
"Expected 2 pieces. Additional pieces discarded in 6 rows [414, 771, 1109, 1442, 1505, 1517]."
Warning message:
"Expected 2 

$G05
# A tibble: 2,178 x 4
   precinctID precinct                   G05DHOD G05RHOD
   <chr>      <chr>                        <int>   <int>
 1 001        "ARLINGTON"                    616       0
 2 001        "CENTRAL"                        0     773
 3 001        "CHESAPEAKE"                     0     569
 4 001        "DEAN"                         497    1090
 5 001        "EAST"                           0     406
 6 001        "EAST WARD"                    400     164
 7 001        "EAST WARD PRECINCT"           457     530
 8 001        "EMANUEL A. M. E. CHURCH "     701       0
 9 001        "FIRST"                          0     643
10 001        "FIRST WARD"                  1668     524
# ... with 2,168 more rows

$G07
# A tibble: 2,266 x 4
   precinctID precinct                   G07DHOD G07RHOD
   <chr>      <chr>                        <int>   <int>
 1 001        "ARLINGTON"                    349       0
 2 001        "CENTRAL"                        0     627
 3 001

In [26]:
df_shp <- st_read("C:/Users/madie/OneDrive/data/VA-2017/VA_precincts/VA_precincts.shp")
head(df_shp, 2)

Reading layer `VA_precincts' from data source `C:\Users\madie\OneDrive\data\VA-2017\VA_precincts\VA_precincts.shp' using driver `ESRI Shapefile'
Simple feature collection with 2439 features and 56 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: -373531.2 ymin: 60026.37 xmax: 380257.5 ymax: 385298.6
projected CRS:  Lambert_Conformal_Conic


precinct       locality              loc_prec                            
1 Glenkirk       Prince William County Prince William County Glenkirk      
2 Buckland Mills Prince William County Prince William County Buckland Mills
  district                 G18DHOR G18DSEN G18OHOR G18OSEN G18RHOR G18RSEN
1 Congressional District 1 980.0   1044.0  0.0     32.0    950.0   856.0  
2 Congressional District 1 938.0   978.0   0.0     31.0    766.0   701.0  
  geometry                       ... ASIANVAP NHPIVAP      OTHERVAP X2MOREVAP
1 MULTIPOLYGON (((163199.1 30... ... 401      1.000000e+00 4        41       
2 MULTIPOLYGON (((162078.8 30... ... 449      1.190457e-07 6        73       
  CD_12 CD_16 HDIST_11 HDIST_REM SENDIST geometry                      
1 1     1     13       13        13      MULTIPOLYGON (((163199.1 30...
2 1     1     13       13        13      MULTIPOLYGON (((162078.8 30...

In [29]:
df_shp_no_geo <- df_shp %>% st_set_geometry(NULL)
write.csv(df_shp_no_geo, "2017-population-precincts.csv")

Since I'm planning on matching the fields called `precinct` in the two different data frames, I need to make the column in `df_shp` match `df_votes` by making it all caps. 

In [8]:
df_shp$precinct = toupper(df_shp$precinct)
head(df_shp, 2)

precinct       locality              loc_prec                            
1 GLENKIRK       Prince William County Prince William County Glenkirk      
2 BUCKLAND MILLS Prince William County Prince William County Buckland Mills
  district                 G18DHOR G18DSEN G18OHOR G18OSEN G18RHOR G18RSEN
1 Congressional District 1 980.0   1044.0  0.0     32.0    950.0   856.0  
2 Congressional District 1 938.0   978.0   0.0     31.0    766.0   701.0  
  geometry                       ... ASIANVAP NHPIVAP      OTHERVAP X2MOREVAP
1 MULTIPOLYGON (((163199.1 30... ... 401      1.000000e+00 4        41       
2 MULTIPOLYGON (((162078.8 30... ... 449      1.190457e-07 6        73       
  CD_12 CD_16 HDIST_11 HDIST_REM SENDIST geometry                      
1 1     1     13       13        13      MULTIPOLYGON (((163199.1 30...
2 1     1     13       13        13      MULTIPOLYGON (((162078.8 30...

In [9]:
# remove precinctID since it's actually not a unique identifier
remove_id <- function(x) {
    x %>%
        subset(select = -precinctID)
}
df_votes <- df_votes %>% lapply(remove_id)
print(df_votes)

$G05
# A tibble: 2,178 x 3
   precinct                   G05DHOD G05RHOD
   <chr>                        <int>   <int>
 1 "ARLINGTON"                    616       0
 2 "CENTRAL"                        0     773
 3 "CHESAPEAKE"                     0     569
 4 "DEAN"                         497    1090
 5 "EAST"                           0     406
 6 "EAST WARD"                    400     164
 7 "EAST WARD PRECINCT"           457     530
 8 "EMANUEL A. M. E. CHURCH "     701       0
 9 "FIRST"                          0     643
10 "FIRST WARD"                  1668     524
# ... with 2,168 more rows

$G07
# A tibble: 2,266 x 3
   precinct                   G07DHOD G07RHOD
   <chr>                        <int>   <int>
 1 "ARLINGTON"                    349       0
 2 "CENTRAL"                        0     627
 3 "CHESAPEAKE"                     0     704
 4 "DEAN"                         383     975
 5 "EAST"                         339     241
 6 "EAST WARD"                    626       

In [10]:
# remove all the unnecessary election results cols from df_shp
rem_cols <- names(df_shp) %in% c('G18DHOR','G18DSEN','G18OHOR','G18OSEN','G18RHOR','G18RSEN','G17DHOD', 'G17RHOD', 
                                 'G17OHOD', 'G17DGOV','G17DLTG','G17DATG', 'G17RGOV','G17RLTG','G17RATG','G17OGOV',
                                 'G16DPRS','G16RPRS','G16OPRS','G16DHOR','G16RHOR','G16OHOR')
df_shp <- df_shp[!rem_cols]
colnames(df_shp)

[1] "precinct"  "locality"  "loc_prec"  "district"  "TOTPOP"    "NH_WHITE" 
 [7] "NH_BLACK"  "NH_AMIN"   "NH_ASIAN"  "NH_NHPI"   "NH_OTHER"  "NH_2MORE" 
[13] "HISP"      "H_WHITE"   "H_BLACK"   "H_AMN"     "H_ASIAN"   "H_NHPI"   
[19] "H_OTHER"   "H_2MORE"   "VAP"       "HVAP"      "WVAP"      "BVAP"     
[25] "AMINVAP"   "ASIANVAP"  "NHPIVAP"   "OTHERVAP"  "X2MOREVAP" "CD_12"    
[31] "CD_16"     "HDIST_11"  "HDIST_REM" "SENDIST"   "geometry"

In [16]:
df_net_1 <- df_shp %>%
    left_join(df_votes$G05, by = "precinct") %>%
    left_join(df_votes$G07, by = "precinct") %>%
    left_join(df_votes$G09, by = "precinct") %>%
    left_join(df_votes$G11, by = "precinct")
print(df_net_1)

Simple feature collection with 151017 features and 42 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: -373531.2 ymin: 60026.37 xmax: 380257.5 ymax: 385298.6
projected CRS:  Lambert_Conformal_Conic
First 10 features:
         precinct              locality                             loc_prec
1        GLENKIRK Prince William County       Prince William County Glenkirk
2  BUCKLAND MILLS Prince William County Prince William County Buckland Mills
3       LIMESTONE Prince William County      Prince William County Limestone
4          MULLEN Prince William County         Prince William County Mullen
5          SUDLEY Prince William County         Prince William County Sudley
6      BEN LOMOND Prince William County     Prince William County Ben Lomond
7       STONEWALL Prince William County      Prince William County Stonewall
8       STONEWALL Prince William County      Prince William County Stonewall
9        BEALETON       Fauquier County             Fauquier C

In [19]:
df_net_2 <- df_shp %>%
    left_join(df_votes$G13, by = "precinct") %>%
    left_join(df_votes$G15, by = "precinct") %>%
    left_join(df_votes$G17, by = "precinct") %>%
    left_join(df_votes$G19, by = "precinct")
print(df_net_2)

Simple feature collection with 121530 features and 42 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: -373531.2 ymin: 60026.37 xmax: 380257.5 ymax: 385298.6
projected CRS:  Lambert_Conformal_Conic
First 10 features:
         precinct              locality                             loc_prec
1        GLENKIRK Prince William County       Prince William County Glenkirk
2  BUCKLAND MILLS Prince William County Prince William County Buckland Mills
3       LIMESTONE Prince William County      Prince William County Limestone
4          MULLEN Prince William County         Prince William County Mullen
5          SUDLEY Prince William County         Prince William County Sudley
6      BEN LOMOND Prince William County     Prince William County Ben Lomond
7       STONEWALL Prince William County      Prince William County Stonewall
8        BEALETON       Fauquier County             Fauquier County Bealeton
9            RUBY       Stafford County                 Staffo

In [22]:
# now i need to remove all of the cols except precinct, and the election results in df_net_2 before I can merge into df_net_1
df_net_2 <- df_net_2 %>% subset(select = c("precinct", "G13DHOD", "G13RHOD",
                                          "G15DHOD", "G15RHOD",
                                          "G17DHOD", "G17RHOD",
                                          "G19DHOD", "G19RHOD")) %>% st_set_geometry(NULL)
print(df_net_2)

                         precinct G13DHOD G13RHOD G15DHOD G15RHOD G17DHOD
1                        GLENKIRK     478     628     325     493     580
2                  BUCKLAND MILLS     819    1028     224     361     536
3                       LIMESTONE     632     769     380     598     859
4                          MULLEN     655     366     291     179     703
5                          SUDLEY     345     429     271     370     410
6                      BEN LOMOND     545     326     313     256     686
7                       STONEWALL       0     447       0     166     133
8                        BEALETON     397     782       0     651     488
9                            RUBY     259     743     162     569     363
10                      ROCK HILL     354     730     234     561     577
11                      STEFANIGA     488    1094     338     792     741
12                      ROSEVILLE     549     948     341     692     835
13                       QUANTICO     

In [23]:
# now join df_net_2 into df_net_1
df_net <- full_join(df_net_1, df_net_2, by = "precinct")
print(df_net)

ERROR: Error: cannot allocate vector of size 43.2 Gb


Ok, so it looks like I need to resolve these memory problems. However, I've since determined that I will need a unique shapefile for every single years, so this project has become somewhat obsolete. Oh well, that's how it goes. 

In [ ]:
st_write(df_net, "C:/Users/madie/OneDrive/data/merged-shp-official-VA/VA_precincts_05-19.shp")